# Análise exploratória dos títulos presentes na Netflix

O objetivo desse mini-projeto é a visualização e compreensão do conjunto de dados em questão que representa os filmes e programas de TV disponíveis na Netflix.  

Esse conjunto de dados foi obtido na plataforma Kaggle:


*   https://www.kaggle.com/shivamb/netflix-shows





Para que seja possível realizar essa análise, será necessário utilizar um conjunto de ferramentas que auxilirão em todo processo. São elas: 

* Pandas: biblioteca que possibilitará a manipulação e análise dos dados. 

* Plotly, Matplotlib, Seaborn: bibliotecas gráficas para visualização de dados.






In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from matplotlib.ticker import FormatStrFormatter 
import plotly.graph_objects as go
import plotly.express as exp 

O primeiro passo a ser dado é fazer a leitura da nossa base de dados. Nesse caso, ela foi disponibilizada no formato '.csv' e a utilizaremos de forma local, ou seja, é necessário fazer o upload do arquivo antes de executar as células do colab. 

In [ ]:
netflix_data = pd.read_csv('../input/netflix-shows/netflix_titles.csv')
netflix_data

Após visualizar rapidamente como é composto nosso conjunto de dados, podemos extrair algumas informações sobre ele rapidamente.

In [ ]:
netflix_data.columns

## O que significa cada coluna?

* ```show_id```: código de identificação de cada título.
* ```type```: identifica qual o tipo do título, se é um filme ou um programa de tv.
* ```title```: nome do título.
* ```director```: diretor ou diretores do filme ou programa de TV.
* ```cast```: elenco.
* ```country```: país origem.
* ```date_added```: data em que foi adicionado a Netflix.
* ```release_year```: ano de lançamento.
* ```rating```: classificação do título
* ```duration```: duração em minutos para o caso dos filmes e em temporadas para os programas de TV. 
* ```listed_in```: categoria do título.
* ```description```: sinopse. 


In [ ]:
netflix_data.info()

## Identificando valores nulos no conjunto de dados 

In [ ]:
#Identificando quantos valores núlos cada coluna possui
netflix_data.isnull().sum()

# Preparação dos dados

Nessa etapa, realizaremos transformações que facilitarão a análise dos dados, dentre elas a remoção de colunas desnecessárias, tratamento de strings e datas, etc.

A primeira mudança constituirá na remoção da coluna ```show_id```, isso porque não será necessário manter identificadores únicos para esse caso. 



In [ ]:
netflix_data = netflix_data.drop(labels=['show_id'], axis=1)

A coluna ```date_added``` que representa quando o título foi adicionado no catalogo da Netflix, está no formato de ```string``` e para tornar esse campo mais fácil de usar, realizaremos a conversão para o tipo ```datetime```, usando da função do pandas: ```to_datetime()```

Além disso, isolaremos tanto o mês quanto o ano em novas colunas do DataFrame.




In [ ]:
## add new features in the dataset
netflix_data["date_added"] = pd.to_datetime(netflix_data['date_added'])
netflix_data['year_added'] = netflix_data['date_added'].dt.year
netflix_data['month_added'] = netflix_data['date_added'].dt.month

Outras duas mudanças que podem ser feitas dizem respeito a coluna ```duration```, nela temos a descrição do tempo sendo feita com valores em minutos ou com a quantidade de temporadas, por exemplo, "90 min" ou "3 Seasons", porém, em termos de analise possuir esses dados no formato de ```string``` não é vantajoso. E para tratar disso, utilizaremos os recursos: 
 * ```apply()```: Possibilita aplicar uma função em um eixo do dataframe;
 * ```split()```: Vai dividir a string quando encontrar o caracter passado; 

A primeira mudança diz respeito a criação de uma nova coluna: ```season_count```, ela conterá apenas o valor númerico correspondente a  quantidade de temporadas de cada série ou programa de TV. 

E a segunda mudança é a remoção da unidade "min" na coluna ```duration```


In [ ]:
netflix_data['season_count'] = netflix_data.apply(
    lambda x : x['duration'].split(" ")[0] if "Season" in x['duration'] else "",
     axis = 1)

netflix_data['duration'] = netflix_data.apply(
    lambda x : x['duration'].split(" ")[0] if "Season" not in x['duration'] else "",
     axis = 1)

Após realizadas essas transformações os valores das linhas nas colunas ```duration``` e  ```season_count``` ainda são ```strings``` e para que possamos mudar isso, utilizaremos a função do pandas: ```to_numeric()``` que será responsável por transformar toda a coluna em valores númericos.

Além disso, as posições vazias dessa coluna estão definidas como `strings` vazias e para facilitar o tratamento desses valores ausentes utilizaremos a função `replace()` para substituir esses valores por `NaN` e com isso poderemos remover os valores faltantes de forma mais simples. 

In [ ]:
netflix_data['duration'] = pd.to_numeric(netflix_data['duration'].replace("", np.NaN))
netflix_data['season_count'] = pd.to_numeric(netflix_data['season_count'].replace("", np.NaN))


In [ ]:
netflix_data.head()

# Divisão do conteúdo da Netflix



In [ ]:
movie = netflix_data.filter(items=['type', 'year_added']).loc[netflix_data['type'] == 'Movie'].dropna()
tv_show = netflix_data.filter(items=['type', 'year_added']).loc[netflix_data['type'] == 'TV Show'].dropna()

In [ ]:
fig = go.Figure(data=[go.Pie(labels=['Filmes', 'Programas de TV'], values= [movie['type'].count(),
                                                                            tv_show['type'].count()],
                             textinfo='label+percent')], layout=go.Layout(title="Divisão do conteúdo"))
fig.show()

# Crescimento dos filmes e programas de TV ao longo dos anos.

O crescimento será analisado com base no ano em que foi adicionado ao catalogo da Netflix (`date_added`)

In [ ]:
moviesByYearAdded = movie['year_added'].value_counts().reset_index().rename(columns={'year_added': 'count', 'index': 'year'})
moviesByYearAdded = moviesByYearAdded.sort_values(by=['year'], ascending=True)

tvShowByYearAdded = tv_show['year_added'].value_counts().reset_index().rename(columns={'year_added': 'count', 'index': 'year'})
tvShowByYearAdded = tvShowByYearAdded.sort_values(by=['year'], ascending=True)

In [ ]:
moviePlot = go.Scatter(x=moviesByYearAdded['year'], y=moviesByYearAdded['count'], name = "Movie")
tvshowPlot = go.Scatter(x=tvShowByYearAdded['year'], y = tvShowByYearAdded['count'], name = "TV Show")


layout = go.Layout(
    title = "Crescimento dos títulos baseado no seu tipo ao longo dos anos",
    xaxis_title = "Anos",
    yaxis_title = "Quantidade",
)

fig = go.Figure(data = [moviePlot, tvshowPlot] , layout = layout)
fig.show()


# Como são as durações dos filmes e dos programas de TV na Netflix?


### Filmes

In [ ]:
duration_movie = netflix_data['duration'].dropna()

In [ ]:
duration_movie.describe(include='all')

In [ ]:
print("Duração total: ", round(duration_movie.sum()/60,2), "horas")

In [ ]:
fig = exp.histogram(netflix_data['duration'], x="duration", title = "Quantidade de titulos por intervalo de duração" )
fig.update_layout(
    xaxis_title = "duração",
    yaxis_title = "quantidade",
)
fig.show()

### Programas de TV

In [ ]:
tvshow_duration = netflix_data['season_count'].dropna()

In [ ]:
tvshow_duration.describe(include='all')

In [ ]:
print("Duração total: ", round(tvshow_duration.sum(),2), "temporadas")

In [ ]:
fig = exp.histogram(netflix_data['season_count'], x = "season_count",
                    title = "Quantidade de programas de TV por número de temporadas")
fig.update_layout(
    xaxis_title = "temporadas",
    yaxis_title = "quantidade",
)
fig.show()

# Distribuição dos títulos com base na classificação

In [ ]:
titlesByRating = netflix_data.copy()
titlesByRating = titlesByRating['rating'].value_counts().reset_index().rename(columns={'index': 'rating', 'rating': 'count'})
titlesByRating
fig, ax = plt.subplots(figsize=(10,10))
sns.set_style(style="darkgrid")
ax = sns.barplot(x=titlesByRating['rating'], y = titlesByRating['count'], data=titlesByRating)
ax.tick_params(labelsize=12) #Ajustando ticklabels para uma fonte maior. 
ax.set_xlabel("Rating", fontsize = 12)
ax.set_ylabel("Quantidade", fontsize = 12)
fig.suptitle("Quantidade de títulos por classificação", fontsize = 16, y = 0.92)
plt.show()

# Quantidade de títulos por país

In [ ]:
'''
    Essa função irá "quebrar" as Series que possuem múltiplos valores em suas linhas, transformando toda linha em valor único
    Dessa forma será possível análisar, por exemplo, todos os possíveis diretores, atores, atrizes, etc.
'''
def transformColumn(serie):
  new_serie = []
  for i in serie: 
   if (len(i) > 1): #Verifica se há mais de um autor por título
     for j in i:
       new_serie.append(j.strip()) #Insere individualmente cada diretor re
   else:
     new_serie.append(i[0])

  new_serie = pd.Series(data=new_serie)
  return new_serie


In [ ]:
netflixDataCopy = netflix_data.copy()
titlesByCountry = netflix_data['country'].dropna()

titlesByCountry = titlesByCountry.apply(lambda x: x.split(","))
titlesByCountry = transformColumn(titlesByCountry)
titlesByCountry = titlesByCountry.value_counts().reset_index().rename(columns={"index": "country", 0: "count"})[0:20]



In [ ]:
fig = exp.bar(x=titlesByCountry['count'], y = titlesByCountry['country'], orientation='h',
              data_frame=titlesByCountry,
              color='country', title="Títulos por país")
fig.update_layout(dict(xaxis_title="quantidade", yaxis_title="países",
                      ))
fig.show()

# Como foi o crescimentos dos títulos em um conjunto de países? 

Para responder essa pergunta é necessário definir um conjunto de países e uma escala de tempo para a análise. Serão avaliados 4 países dentro do intervalo de tempo de 2000 até 2020. Sendo que será considerado o ano de lançamento.

E colocaremos em foco 4 países:

* Estados Unidos
* India
* Reino Unido
* Brasil 

In [ ]:
increaseTitlesNetflix = netflix_data.copy()
increaseTitlesNetflix = increaseTitlesNetflix.filter(items=['country', 'release_year'])
increaseTitlesNetflix = increaseTitlesNetflix.loc[(increaseTitlesNetflix['country'].isin(['Brazil', 'United States',
                                                                                          'United Kingdom',
                                                                                          'India']))]
                                                                                        

increaseTitlesNetflix = increaseTitlesNetflix.dropna()
increaseTitlesNetflix = increaseTitlesNetflix.loc[increaseTitlesNetflix['release_year'] >= 2000]

In [ ]:
brazilianTitlesByReleaseYear = increaseTitlesNetflix.loc[increaseTitlesNetflix['country'] == 'Brazil']

brazilianTitlesByReleaseYear = brazilianTitlesByReleaseYear['release_year'].value_counts().reset_index().rename(
    columns={'index': 'release_year', 'release_year': 'count'})

brazilianTitlesByReleaseYear = brazilianTitlesByReleaseYear.sort_values(by=['release_year'], ascending=True)


In [ ]:
UnitedStatesTitlesByReleaseYear = increaseTitlesNetflix.loc[increaseTitlesNetflix['country'] == 'United States']

UnitedStatesTitlesByReleaseYear = UnitedStatesTitlesByReleaseYear['release_year'].value_counts().reset_index().rename(
    columns={'index': 'release_year', 'release_year': 'count'})
UnitedStatesTitlesByReleaseYear = UnitedStatesTitlesByReleaseYear.sort_values(by=['release_year'], ascending=True)


In [ ]:
UKTitlesByReleaseYear = increaseTitlesNetflix.loc[increaseTitlesNetflix['country'] == 'United Kingdom']

UKTitlesByReleaseYear = UKTitlesByReleaseYear['release_year'].value_counts().reset_index().rename(
    columns={'index': 'release_year', 'release_year': 'count'})
UKTitlesByReleaseYear = UKTitlesByReleaseYear.sort_values(by=['release_year'], ascending=True)

In [ ]:
indiaTitlesByReleaseYear = increaseTitlesNetflix.loc[increaseTitlesNetflix['country'] == 'India']

indiaTitlesByReleaseYear = indiaTitlesByReleaseYear['release_year'].value_counts().reset_index().rename(
    columns={'index': 'release_year', 'release_year': 'count'})
netflix_data
indiaTitlesByReleaseYear = indiaTitlesByReleaseYear.sort_values(by=['release_year'], ascending=True)


In [ ]:
unitedStatesPlot = go.Scatter(x = UnitedStatesTitlesByReleaseYear['release_year'],
                                     y =UnitedStatesTitlesByReleaseYear['count'],
                                     marker=dict(color='blue'),name = "Estados Unidos", mode = 'lines')

brazilPlot = go.Scatter(x = brazilianTitlesByReleaseYear['release_year'],
                                     y =brazilianTitlesByReleaseYear['count'],
                                     marker=dict(color='purple'),name = "Brasil", mode ='lines')

ukPlot = go.Scatter(x = UKTitlesByReleaseYear['release_year'],
                                     y = UKTitlesByReleaseYear['count'],
                                     marker=dict(color='red'),  name = "Reino Unido", mode = 'lines')

indiaPlot = go.Scatter(x = indiaTitlesByReleaseYear['release_year'],
                                     y = indiaTitlesByReleaseYear['count'],
                                     marker=dict(color='gold'), name = "India", mode = 'lines')

dataPlot = [unitedStatesPlot, brazilPlot, ukPlot, indiaPlot]

layout = go.Layout(
    title = 'Crescimento dos títulos por país ao longo dos anos',
    xaxis_title = "Anos",
    yaxis_title = "Quantidade", 
    xaxis = dict(tickvals=[2000, 2002, 2005, 2008, 2010, 2012, 2015, 2018, 2020]),
)
fig = go.Figure(data=dataPlot, layout=layout)

fig.show()

# Quantidade de títulos por diretor


In [ ]:
#Preparação da lista de diretores
directors = netflix_data['director'].dropna()

directors = directors.apply(lambda x : x.split(","))

#Responsável por transformar os elementos que são listas em novos elementos
new_serie = transformColumn(directors)
new_serie = new_serie.value_counts()

directors_df = pd.DataFrame()
directors_df['director'] = new_serie
directors_df = directors_df.reset_index().rename(columns={'director': 'count', "index": "director"})
directors_df = directors_df[0:30]


In [ ]:
bar1 = go.Bar(x = directors_df['director'], y=directors_df['count'],  orientation='v', 
               marker={'color': 'coral'})

layout = go.Layout(
    title="Quantidade de títulos por diretor",
    xaxis_title= "Diretores",
    yaxis_title = "Quantidade",
)

fig = go.Figure(data= bar1, layout=layout)
fig.show()

# Quais são os gêneros presentes na Netflix?


In [ ]:
gender = netflix_data['listed_in']
gender = gender.apply(lambda x: x.split(","))
gender = transformColumn(gender)
gender = gender.value_counts().reset_index().rename(columns={"index": "gender", 0: "count"})[0:20]


In [ ]:
plot = go.Bar(x=gender['count'], y=gender['gender'], orientation='h')
layout = go.Layout(
    title = "Títulos por gênero",
    xaxis_title = "quantidade",
    yaxis_title = "gêneros",
  
)
fig = go.Figure(data=plot, layout=layout)
fig.update_yaxes(autorange='reversed')
fig.show()

# Atores com mais filmes na Netflix


In [ ]:
def actorsByCountryTrace(country):
  actorsData = netflix_data.copy()
  aux = pd.Series(dtype='object')
  actorsData = actorsData.filter(items=['cast', 'country', 'type']).dropna()
  
  actorsData = actorsData.loc[(actorsData['country'] == country) & (actorsData['type'] == "Movie")]
 
  newActorsData = actorsData['cast'].apply(lambda x: x.split(","))
  
  newActorsData = transformColumn(newActorsData)
  newActorsData = newActorsData.value_counts().reset_index().rename(columns={"index": "actor", 0:'count'})[0:20]
  
  return go.Bar(x=newActorsData['count'], y = newActorsData['actor'],
                orientation='h',marker=dict(color="#355070"), name = "".format(country))

In [ ]:
traces = []
countries = ['United States', 'United Kingdom', 'India', 'Japan', 'Canada']
COLORS = ['moccasin', 'mediumpurple', 'darkblue', 'deepskyblue', 'antiquewhite']

for country in countries: 
  traces.append(actorsByCountryTrace(country))

In [ ]:
from plotly.subplots import make_subplots
fig = make_subplots(rows = 2, cols= 3, subplot_titles=countries, horizontal_spacing=0.2, vertical_spacing=0.1)

fig.add_trace(traces[0], row=1, col=1)
fig.add_trace(traces[1], row=1, col=2)
fig.add_trace(traces[2], row=1, col=3)
fig.add_trace(traces[3], row=2, col=1)
fig.add_trace(traces[4], row=2, col=2)

fig.update_layout(height=1200, showlegend=False, title='Top 20 atores com mais filmes')
fig.update_yaxes(autorange='reversed')
fig.show()